In [1]:
import nltk
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/suveyk/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/suveyk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/suveyk/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# Importing Required Libraries
import re
from nltk.corpus import wordnet
import itertools

# List of Keywords
list_words=['hello','hey','hi','bye','suicide','kill','die', 'dark','scared','vulnerable','anxiety','depression','want','feel','think', 'was','thought','thinking']

list_syn={}
for word in list_words:
    synonyms=[]
    for syn in wordnet.synsets(word): 
        for lem in syn.lemmas(): 
            lem_name = re.sub('[^a-zA-Z0-9 \n\.]', ' ', lem.name())
            synonyms.append(lem_name)
    list_syn[word]=set(synonyms)

# Building Dictionary of Intents and Keywords
keywords={}
keywords_dict={}

# Creating a list of synonyms with multiple related words
new_list_syn_greetings = itertools.chain(list_syn['hello'], list_syn['hey'], list_syn['hi'])
new_list_syn_goodbyes = list_syn['bye']
new_list_syn_depression = itertools.chain(list_syn['suicide'], list_syn['kill'], list_syn['die'],list_syn['scared'],list_syn['vulnerable'],list_syn['depression'],list_syn['anxiety'])
new_list_syn_thinking = itertools.chain(list_syn['think'], list_syn['was'], list_syn['thought'], list_syn['thinking'])

list_keywords = {
    'greetings': new_list_syn_greetings,
    'goodbyes': new_list_syn_goodbyes,
    'depression': new_list_syn_depression, 
    'automaticThought': new_list_syn_thinking
    }

for keyword in list_keywords:
    keywords[keyword]=[]
    for synonym in list(list_keywords[keyword]):
        keywords[keyword].append('.*\\b'+synonym+'\\b.*') 

    for intent, keys in keywords.items():
        keywords_dict[intent]=re.compile('|'.join(keys))

#Building a Dictionary of Responses
responses={
    'hello':'Hey, how do you feel today?',
    'depression':'If you are scared and anxious, please contact ABC - +00 0000',
    'goodbyes':'Bye, Thank you reaching out. I hope you have a good day!',
    'thinking': 'What were your feelings like, and what prompted your current thoughts?'
}

def get_response(msg):
    matched_intent = None 
    for intent,pattern in keywords_dict.items():
        #Search Input based on Keywords
        if re.search(pattern, msg): 
            #Select Instant based on Keyword
            matched_intent = intent  
    
    if matched_intent in responses:
        key = matched_intent
        return responses[key]
    return 'none'

In [60]:
from tkinter import *
from nltk.sentiment import SentimentIntensityAnalyzer
import pickle

BG_DARK = "#15191d"
BG_LIGHTER = "#212529"
BG_GRAY = "#9dc5ed"
BG_COLOR = "white"
TEXT_COLOR = "white"

FONT = "Calibri 14"
FONT_BOLD = "Calibri 23 bold"

bot_name = "Angela"
sample_responses = [ "I'm sorry you're feeling this way. Was there anything in particular that triggered this feeling? Please give a thorough explanation of the circumstance.","What were you thinking at the time that contributed to your current predicament?","Please tell me how intense your thoughts and feelings were: 0 -  low, 1 - medium, and 2 - high!","Why do you believe on your thoughts being genuine and real?","Is there substantial evidence to demonstrate that your thoughts are incorrect?","Taking the scenario into consideration, please think carefully and let me know if there is another way to think about and handle the situation. Please provide an example of how it could have gone differently.","How would you rate your mood now? 0 - Negative, 1 - Neutral 2 - Positive",]


suggest_exercise = "I'm sorry, I know it's hard. Please practice mindful meditation, reframe your negative thoughts, and visualize the best parts of your day!"           

class interface:
    
    def __init__(self):
        self.window = Tk()
        self.setup_main_window()
        self.msg_entry.delete(0, END)
        self.insert_angela_message("Hello!\
            \nHey, I'm Angela!\
            \nHow are you?")
        self.step = 0 #Steps that Indicate the Stage of the Self-Assessment
        self.sia = SentimentIntensityAnalyzer()
        self.automatic_thougth = None
        self.cognitive_distortion = None
        self.pre_mood_rating = None
        self.post_mood_rating = None

    def run(self):
        self.window.mainloop()

    #Interface    
    def setup_main_window(self):
        self.window.title("Self-Assessment ChatBot based on Cognitive Behavioural Therapy")
        self.window.resizable(width=False, height=False) 
        self.window.configure(width=1366, height=768, bg=BG_DARK) 
        
        # TOP LABEL
        head_label = Label(self.window, 
                           bg=BG_DARK, 
                           fg=TEXT_COLOR, 
                           text="Angela", 
                           font=FONT_BOLD, 
                           pady=15)
        head_label.place(relwidth=1)
        
        #BOTTOM LABEL
        bottom_label = Label(self.window, bg=BG_DARK, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        #TEXT WIDGET
        self.text_widget = Text(self.window, 
                                width=20,
                                height=2, 
                                bg=BG_LIGHTER, 
                                fg=TEXT_COLOR, 
                                font=FONT, 
                                padx=25, 
                                pady=25)
        self.text_widget.place(relheight=0.745, relwidth=0.97, rely=0.08, relx=0.015)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        #SCROLL BAR
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        #MESSAGE ENTRY
        self.msg_entry = Entry(bottom_label, bg="#262626", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.018, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self.on_enter_pressed)
        
        #SEND BUTTON
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY, command=lambda: self.on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.018, relheight=0.06, relwidth=0.22)
    
    def on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self.insert_message(msg, "You")
    
    def insert_message(self, msg, sender):
        if not msg:
            return   
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        if (self.step == 3 or self.step == 7): # Check mood rating
            self.insert_angela_message(self.check_rating(msg))
            if (self.step == 7 and (self.post_mood_rating - self.pre_mood_rating) >= 0):
                self.insert_angela_message(suggest_exercise)
                self.step = 0
        else:
            angela_response = get_response(msg)
            if (angela_response == 'Hello! So, tell me how have you been feeling?' or 
                angela_response == 'Bye. Hope I\'ve been helpful.'):
                self.step = 0
            if (angela_response == 'none'):
                if (self.step == 0 and self.sia.polarity_scores(msg)["compound"] > 0):
                    self.insert_angela_message("Yayy, this is great!")
                    self.step = 0
                    
                else:
                    self.insert_angela_message(sample_responses[self.step])
                    if (self.step == 1): 
                        self.automatic_thought = msg
                    self.step = self.step + 1           
            else: 
                self.insert_angela_message(angela_response)
                    
        self.text_widget.see(END)
          
    def insert_angela_message(self, msg):
        msg1 = f"{bot_name}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
    
    #Check if user input is valid - 0, 1, 2
    def check_rating(self, msg):
        angela_response = "Please Enter 0 (or) 1 (2) 2 based on the question."
        if (msg.isnumeric()):
                rating = int(msg)
                if (rating == 0 or rating == 1 or rating == 2):
                    if (self.step == 7):
                        angela_response = self.detect_cognitive_distortion()
                        self.post_mood_rating = rating
                    else:
                        angela_response = sample_responses[self.step]
                        self.step += 1
                        self.pre_mood_rating = rating
        return angela_response

    #Identify Potential Cognitive Distortion with Trained Model
    def detect_cognitive_distortion(self):
        pickled_model = pickle.load(open('cbt-predict.pkl', 'rb'))
        self.cognitive_distortion = pickled_model.predict([self.automatic_thought])[0]
        if (self.cognitive_distortion == 0):
            msg = "You may be experiencing a Cognitive Distortion! Please refer to this link to learn more: https://www.health.harvard.edu/blog/how-to-recognize-and-tame-your-cognitive-distortions-202205042738"
            return msg
     
if __name__ == "__main__":
    app = interface()
    app.run()